In [1]:
import os
import librosa
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

train_audio_path = '../data/'
labels=["house", "left", "six", "tree", "zero"]

all_wave = []
all_label = []
for label in labels:
    print(label)
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

house
left
six
tree
zero


In [2]:
import numpy as np

le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)

y=np_utils.to_categorical(y, num_classes=len(labels))
all_wave = np.array(all_wave).reshape(-1,8000,1)

In [4]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

base_model = load_model('bestModel_set1_80.hdf5')
base_model.trainable = False
# print(base_model.summary())

model2= Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
# print(model2.summary())

x = model2.output
x = Dense(len(labels), activation='softmax', name='dense_2')(x)

final_model = Model(inputs=model2.input, outputs=x)
final_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8000, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 7988, 8)           112       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2662, 8)           0         
_________________________________________________________________
dropout (Dropout)            (None, 2662, 8)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2652, 16)          1424      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 884, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 884, 16)          

In [5]:
x_train, x_test, y_train, y_test = train_test_split(np.array(all_wave),np.array(y),stratify=y,
                                            test_size = 0.8,random_state=777,shuffle=True)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('bestModel_temp.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

final_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = final_model.fit(x_train, y_train, epochs=5, callbacks=[es,mc], batch_size=32, validation_data=(x_test,y_test))

Epoch 1/5
61/61 [==============================] - ETA: 0s - loss: 1.6240 - accuracy: 0.3230
Epoch 00001: val_accuracy improved from -inf to 0.54916, saving model to bestModel_set2_20_tlOn.hdf5
61/61 [==============================] - 12s 201ms/step - loss: 1.6240 - accuracy: 0.3230 - val_loss: 1.1992 - val_accuracy: 0.5492
Epoch 2/5
60/61 [============================>.] - ETA: 0s - loss: 1.1599 - accuracy: 0.5203
Epoch 00002: val_accuracy improved from 0.54916 to 0.69258, saving model to bestModel_set2_20_tlOn.hdf5
61/61 [==============================] - 12s 195ms/step - loss: 1.1603 - accuracy: 0.5205 - val_loss: 0.9909 - val_accuracy: 0.6926
Epoch 3/5
60/61 [============================>.] - ETA: 0s - loss: 1.0116 - accuracy: 0.6089
Epoch 00003: val_accuracy improved from 0.69258 to 0.72794, saving model to bestModel_set2_20_tlOn.hdf5
61/61 [==============================] - 12s 196ms/step - loss: 1.0094 - accuracy: 0.6102 - val_loss: 0.9001 - val_accuracy: 0.7279
Epoch 4/5
60/61 